# Google BigQuery Social Listening Data Export

This notebook connects to BigQuery using Google Cloud CLI authentication and exports social media listening data to CSV.

## Prerequisites Setup Guide

### 1. Install Required Tools

Before using this notebook, you need to install:

1. **uv** (Python package manager):
   ```bash
   # On macOS/Linux
   curl -LsSf https://astral.sh/uv/install.sh | sh
   
   # On Windows
   powershell -c "irm https://astral.sh/uv/install.ps1 | iex"
   ```

2. **Google Cloud CLI**:
   - Download from: https://cloud.google.com/sdk/docs/install
   - Follow the installation instructions for your operating system

### 2. Set Up Project Environment

In your terminal (not in this notebook):

```bash
# Navigate to project directory
cd /path/to/google-cloud-proj

# Install Python dependencies
uv sync

# Start Jupyter Notebook with the correct environment
uv run jupyter notebook
```

### 3. Authenticate with Google Cloud

Run this command in your terminal (only needed once per machine):

```bash
# Authenticate with Google Cloud
gcloud auth application-default login

# Set your default project
gcloud config set project sinnia-gnp  # Replace with your project ID
```

### 4. Common Issues and Solutions

- **Import errors**: Make sure you started Jupyter with `uv run jupyter notebook`
- **Authentication errors**: Run `gcloud auth application-default login` in your terminal
- **Permission errors**: Ensure your Google account has BigQuery access for the project

## 0. Virtual Environment Setup

**IMPORTANT**: This notebook requires the proper Python environment with all dependencies installed.

If you're seeing import errors, you need to:
1. Install dependencies using `uv sync` in the terminal
2. Start Jupyter with `uv run jupyter notebook` or `uv run jupyter lab`

The cell below will check if the environment is set up correctly.

In [2]:
# Check if running in the correct environment
import sys
import os

print("Python executable:", sys.executable)
print("Python version:", sys.version)

# Try to import required packages
try:
    import google.cloud.bigquery
    print("✓ google-cloud-bigquery is installed")
except ImportError:
    print("❌ google-cloud-bigquery is NOT installed")
    print("\nTo fix this:")
    print("1. Close this notebook")
    print("2. In your terminal, run: uv sync")
    print("3. Start Jupyter with: uv run jupyter notebook")
    print("\nIf you're already running with uv run, try restarting the kernel.")

try:
    import pandas
    print("✓ pandas is installed")
except ImportError:
    print("❌ pandas is NOT installed")

# Check if we're in a virtual environment
if hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix):
    print("\n✓ Running in a virtual environment")
else:
    print("\n⚠️  Not running in a virtual environment - packages may not be available")

Python executable: /mnt/d/google_cloud_proj/.venv/bin/python3
Python version: 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
✓ google-cloud-bigquery is installed
✓ pandas is installed

✓ Running in a virtual environment


## 1. Environment Setup

In [3]:
# Import required libraries
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime
from pathlib import Path

# Import helper functions from main.py
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
from main import get_bigquery_client, query_to_csv

In [4]:
# Using Google Cloud CLI authentication (Application Default Credentials)
# Make sure you've run: gcloud auth application-default login

project_id = os.getenv('GOOGLE_CLOUD_PROJECT', 'sinnia-gnp')  # Default to sinnia-gnp if not set

# Check if Application Default Credentials are available
try:
    from google.auth import default
    credentials, project = default()
    print("✓ Using Application Default Credentials")
    if project:
        print(f"✓ Default project from gcloud: {project}")
        if not project_id:
            project_id = project
except Exception as e:
    print("❌ Application Default Credentials not found")
    print("   Please run: gcloud auth application-default login")
    print("   Or install gcloud CLI from: https://cloud.google.com/sdk/docs/install")

/mnt/d/google_cloud_proj/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✓ Using Application Default Credentials
✓ Default project from gcloud: sinnia-gnp


## 2. Authentication Setup

Run the cell below to set up Google Cloud authentication. This only needs to be done once per machine.

**Note:** This will open a browser window where you'll need to log in with your Google account that has access to the BigQuery project.

In [5]:
# If Application Default Credentials are not set up, run this cell to authenticate
# This will open a browser window for authentication
import subprocess
import sys

try:
    # Check if ADC exists
    from google.auth import default
    credentials, project = default()
    print("✓ Application Default Credentials already configured")
except:
    print("Setting up Application Default Credentials...")
    print("A browser window will open for authentication.")
    print("Please log in with your Google account that has access to the BigQuery project.\n")
    
    try:
        # Run gcloud auth command
        result = subprocess.run(
            ["gcloud", "auth", "application-default", "login"],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print("\n✓ Authentication successful!")
            print("Application Default Credentials have been saved.")
        else:
            print(f"\n❌ Authentication failed: {result.stderr}")
            print("\nPlease run this command manually in your terminal:")
            print("gcloud auth application-default login")
    except FileNotFoundError:
        print("❌ gcloud CLI not found. Please install it first:")
        print("https://cloud.google.com/sdk/docs/install")
        print("\nOr run this command in your terminal after installing:")
        print("gcloud auth application-default login")

/mnt/d/google_cloud_proj/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✓ Application Default Credentials already configured


In [6]:
# Initialize BigQuery client with Application Default Credentials
try:
    # No need to pass credentials when using ADC
    client = bigquery.Client(project=project_id)
    print(f"✓ Connected to BigQuery project: {client.project}")
except Exception as e:
    print(f"❌ Error connecting to BigQuery: {e}")
    print("   Please ensure you have run: gcloud auth application-default login")
    print("   And that you have access to the project")

/mnt/d/google_cloud_proj/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✓ Connected to BigQuery project: sinnia-gnp


In [7]:
# Check for credentials configuration
# Option 1: Service account file (if you have one)
credentials_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

# Option 2: Use Application Default Credentials (recommended)
# No credentials_path needed - just ensure you've run:
# gcloud auth application-default login

if credentials_path and credentials_path.strip() and os.path.exists(credentials_path):
    print(f"✓ Service account credentials file found: {credentials_path}")
else:
    print("ℹ️ No service account file configured.")
    print("   Using Application Default Credentials instead.")
    print("   Make sure you've run: gcloud auth application-default login")
    credentials_path = None  # Explicitly set to None to use ADC

ℹ️ No service account file configured.
   Using Application Default Credentials instead.
   Make sure you've run: gcloud auth application-default login


In [8]:
# Initialize BigQuery client
try:
    # Use Application Default Credentials by not passing credentials_path
    client = get_bigquery_client(project_id=project_id)
    print(f"✓ Connected to BigQuery project: {client.project}")
except Exception as e:
    print(f"❌ Error connecting to BigQuery: {e}")
    print("   Please check your credentials and project ID")

/mnt/d/google_cloud_proj/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/mnt/d/google_cloud_proj/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Using Application Default Credentials for project: sinnia-gnp
✓ Connected to BigQuery project: sinnia-gnp


## 3. Define and Run Query

In [9]:
# Define the query
query = """
SELECT 
    platform, 
    CAST(created_at AS STRING FORMAT 'YYYY-MM-DD HH24:MI') AS creado, 
    user_id, 
    user, 
    CAST(followers AS STRING) AS seguidores,
    text, 
    CAST(likes_reactions AS STRING) AS likes, 
    CAST(comments AS STRING) AS comentarios, 
    CAST(shares_retweets AS STRING) AS compartidos,
    CAST(engagements AS STRING) AS enganches,
    CAST(views AS STRING) AS vistas
FROM `sinnia-gnp.social_dashboard_table.listening_table_prd` 
WHERE created_at >= '2025-05-12' 
    AND created_at < '2025-05-14'
    AND topic_id = 238
    AND NOT (
        CONTAINS_SUBSTR(text, "Estadio GNP") 
        OR CONTAINS_SUBSTR(text, "Auditorio GNP") 
        OR CONTAINS_SUBSTR(text, "el GNP") 
        OR CONTAINS_SUBSTR(text, "Foro GNP")
    )
"""

print("Query defined successfully")

Query defined successfully


In [10]:
# Run query and get results as DataFrame
print("Running query...")
try:
    df = client.query(query).to_dataframe()
    print(f"✓ Query completed successfully")
    print(f"✓ Retrieved {len(df):,} rows")
    print(f"\nColumns: {', '.join(df.columns)}")
except Exception as e:
    print(f"❌ Error running query: {e}")

Running query...


/mnt/d/google_cloud_proj/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✓ Query completed successfully
✓ Retrieved 151 rows

Columns: platform, creado, user_id, user, seguidores, text, likes, comentarios, compartidos, enganches, vistas


## 4. Preview Data

In [11]:
# Display first few rows
print("\nFirst 5 rows of data:")
df.head()


First 5 rows of data:


,platform,creado,user_id,user,seguidores,text,likes,comentarios,compartidos,enganches,vistas
0,Twitter,2025-05-13 17:12,1221543005485555712,annoying_girl99,141,@PulsoGNP @GNPSeguros Link de la preventa??? E...,0,0,0,None,None
1,Twitter,2025-05-12 23:09,882309131549511681,sarahiiifl,61,"Bueno ya me rendí, véndame un boleto para Bad ...",2,0,0,2,None
2,Facebook,2025-05-12 17:58,1747493702229048,Luli y Gabo,100298,¡Cuidado con asfixias o quemaduras! 🦖🚫🔥 La sup...,7,0,0,7,0
3,Twitter,2025-05-13 01:57,1821256511156580353,tiffpoilard,2,@wolfkeiira @Noyramo @youcantgetnohig El récor...,1,1,0,2,None
4,Twitter,2025-05-12 18:42,1657217841379041282,eusty92,5,@l_og2 ¿Sabes algo del Pulso GNP 2025?,0,0,0,None,None


In [12]:
# Basic statistics
print(f"Total rows: {len(df):,}")
print(f"\nPlatform distribution:")
print(df['platform'].value_counts())
print(f"\nDate range: {df['creado'].min()} to {df['creado'].max()}")

Total rows: 151

Platform distribution:
platform
Twitter     131
Facebook     19
YouTube       1
Name: count, dtype: int64

Date range: 2025-05-12 00:02 to 2025-05-13 23:57


In [ ]:
# Define aggregated query
aggregated_query = """
SELECT 
    DATE_TRUNC(created_at, DAY) AS fecha,
    platform,
    COUNT(DISTINCT activity_id) AS posts,
    SUM(likes_reactions) as likes,
    SUM(comments) AS comments,
    SUM(shares_retweets) AS shares,
    SUM(engagements) AS engagement,
    SUM(views) AS views
FROM `sinnia-gnp.social_dashboard_table.listening_table_prd`
WHERE topic_id = 238 AND created_at >= '2025-01-01'
GROUP BY 1,2
ORDER BY fecha
LIMIT 1000
"""

print("Aggregated query defined successfully")

In [ ]:
# Run aggregated query and get results as DataFrame
print("Running aggregated query...")
try:
    agg_df = client.query(aggregated_query).to_dataframe()
    print(f"✓ Aggregated query completed successfully")
    print(f"✓ Retrieved {len(agg_df):,} rows")
    print(f"\nColumns: {', '.join(agg_df.columns)}")
    
    # Display first few rows
    print("\nFirst 5 rows of aggregated data:")
    print(agg_df.head())
    
except Exception as e:
    print(f"❌ Error running aggregated query: {e}")

In [ ]:
# Export aggregated data to CSV
agg_filename = f'social_listening_aggregated_topic238_{timestamp}.csv'
agg_filepath = os.path.join('data', agg_filename)

# Export to CSV
agg_df.to_csv(agg_filepath, index=False, encoding='utf-8')
print(f"✓ Aggregated data exported to: {agg_filepath}")
print(f"✓ File size: {os.path.getsize(agg_filepath) / 1024:.2f} KB")

## 5. Export to CSV

In [ ]:
# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Generate filename with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'social_listening_topic238_{timestamp}.csv'
filepath = os.path.join('data', filename)

# Export to CSV
df.to_csv(filepath, index=False, encoding='utf-8')
print(f"✓ Data exported to: {filepath}")
print(f"✓ File size: {os.path.getsize(filepath) / 1024 / 1024:.2f} MB")

## 6. Alternative: Direct Query to CSV (for large datasets)

In [ ]:
# Use this method for very large datasets to avoid memory issues
# Uncomment to use:

# large_filename = f'social_listening_topic238_large_{timestamp}.csv'
# large_filepath = os.path.join('data', large_filename)

# # Export in chunks of 10,000 rows
# query_to_csv(client, query, large_filepath, chunk_size=10000)

## Summary

The social listening data has been successfully exported. You can find your CSV file in the `data/` directory.

### Next Steps:
1. Check the exported CSV file in the `data/` folder
2. Modify the date range in the query to export different time periods
3. Adjust the topic_id to query different topics
4. Add additional filters or columns as needed